In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses

(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = tf.pad(x_train, [[0, 0], [2, 2], [2, 2]])/255
x_test = tf.pad(x_test, [[0, 0], [2, 2], [2, 2]])/255
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)

x_val = x_train[-2000:,:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000:,:,:,]
y_train = y_train[:-2000]

In [3]:
model = models.Sequential()
model.add(layers.experimental.preprocessing.Resizing(224, 224, interpolation='bilinear', input_shape=x_train.shape[1:]))
model.add(layers.Conv2D(96, 11, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(256, 5, strides=4, padding='same'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.Activation('relu'))
model.add(layers.MaxPooling2D(3, strides=2))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(384, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv2D(256, 3, strides=4, padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing (Resizing)          (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 56, 56, 96)        34944     
_________________________________________________________________
lambda (Lambda)              (None, 56, 56, 96)        0         
_________________________________________________________________
activation (Activation)      (None, 56, 56, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 256)         614656    
_________________________________________________________________
lambda_1 (Lambda)            (None, 7, 7, 256)         0

In [4]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs=40, validation_data=(x_val, y_val))

Epoch 1/40
907/907 [==============================] - 138s 118ms/step - loss: 0.4115 - accuracy: 0.8546 - val_loss: 0.1137 - val_accuracy: 0.9735
Epoch 2/40
907/907 [==============================] - 107s 118ms/step - loss: 0.0984 - accuracy: 0.9750 - val_loss: 0.0675 - val_accuracy: 0.9840
Epoch 3/40
907/907 [==============================] - 107s 118ms/step - loss: 0.0742 - accuracy: 0.9812 - val_loss: 0.0650 - val_accuracy: 0.9870
Epoch 4/40
907/907 [==============================] - 107s 118ms/step - loss: 0.0598 - accuracy: 0.9856 - val_loss: 0.0496 - val_accuracy: 0.9905
Epoch 5/40
907/907 [==============================] - 107s 118ms/step - loss: 0.0521 - accuracy: 0.9868 - val_loss: 0.0569 - val_accuracy: 0.9890
Epoch 6/40
907/907 [==============================] - 107s 118ms/step - loss: 0.0517 - accuracy: 0.9878 - val_loss: 0.0471 - val_accuracy: 0.9935
Epoch 7/40
907/907 [==============================] - 107s 117ms/step - loss: 0.0378 - accuracy: 0.9905 - val_loss: 0.0641 -

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15, 15))
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])
axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])
plt.show()

In [ ]:
model.evaluate(x_test, y_test)